##Mounting google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importing neccesary library

In [2]:
import numpy as np
import keras
import os
import itertools
import cv2
import matplotlib.pyplot as plt
import time
import pandas as pd
from keras import backend as K
from keras.models import Model
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Dropout
from keras.layers.convolutional import *
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from sklearn.metrics import classification_report
from keras.models import model_from_json
%matplotlib inline

In [4]:
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Gaze dataset/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Gaze dataset/haarcascade_eye.xml')

In [15]:
import glob
import os
Images = os.listdir("/content/drive/MyDrive/Gaze_dataset/test")
Images.sort()

In [ ]:
os.listdir("/content/drive/MyDrive/Gaze_dataset/test")

In [23]:
!ls

haarcascade_eye.xml		       test
haarcascade_frontalface_default.xml    test_eye_cropped
sample_submission.csv		       train
shape_predictor_68_face_landmarks.dat


In [25]:
import dlib

In [3]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords


In [27]:
not_detected = []
for f in Images:
  img = cv2.imread('./test/'+f)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  detector = dlib.get_frontal_face_detector()
  rects = detector(gray, 1)
  predictor = dlib.shape_predictor('/content/drive/MyDrive/Gaze_dataset/shape_predictor_68_face_landmarks.dat')
  for (i, rect) in enumerate(rects):
      shape = predictor(gray, rect)
      shape = shape_to_np(shape)
  cv2.imwrite('/content/Gaze_dataset/test_eye_cropped/'+f.split("/")[-1],img[shape[37][1]-15:shape[40][1]+15,shape[37][0]-15:shape[40][0]+15])

In [53]:
import dlib

In [31]:
data_dir = '/content/drive/MyDrive/Gaze_dataset/train'


BATCH_SIZE = 32
# IMG_SIZE = (256, 256, 3)

train_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            
                                                            )
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=8)



normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 2500 files belonging to 5 classes.
Found 2500 files belonging to 5 classes.
Using 500 files for validation.


In [34]:
!ls

haarcascade_eye.xml		       test
haarcascade_frontalface_default.xml    test_eye_cropped
sample_submission.csv		       train
shape_predictor_68_face_landmarks.dat


In [35]:
train_path = 'train'
test_path  = 'test'
print(os.listdir(train_path))
print(os.listdir(test_path))

['.DS_Store', 'Left_wing_mirror', 'Centerstack', 'Rearview_mirror', 'Forward', 'Right_wing_mirror']
['409.jpg', '376.jpg', '160.jpg', '440.jpg', '101.jpg', '486.jpg', '105.jpg', '357.jpg', '39.jpg', '103.jpg', '237.jpg', '370.jpg', '7.jpg', '477.jpg', '210.jpg', '433.jpg', '280.jpg', '311.jpg', '96.jpg', '362.jpg', '365.jpg', '107.jpg', '299.jpg', '399.jpg', '448.jpg', '6.jpg', '176.jpg', '312.jpg', '507.jpg', '415.jpg', '79.jpg', '21.jpg', '218.jpg', '452.jpg', '439.jpg', '488.jpg', '130.jpg', '346.jpg', '288.jpg', '165.jpg', '494.jpg', '508.jpg', '161.jpg', '476.jpg', '297.jpg', '178.jpg', '135.jpg', '54.jpg', '113.jpg', '364.jpg', '389.jpg', '220.jpg', '447.jpg', '221.jpg', '202.jpg', '468.jpg', '56.jpg', '475.jpg', '344.jpg', '147.jpg', '154.jpg', '32.jpg', '317.jpg', '506.jpg', '360.jpg', '62.jpg', '72.jpg', '145.jpg', '76.jpg', '173.jpg', '314.jpg', '236.jpg', '73.jpg', '195.jpg', '175.jpg', '108.jpg', '382.jpg', '259.jpg', '197.jpg', '235.jpg', '478.jpg', '345.jpg', '226.jpg', '

In [36]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
    validation_split=0.2)

In [37]:
batchSize=64


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=batchSize,
    subset='validation')


Found 2000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


# **Inception v3**

In [38]:
base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(224, 224,3))
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)

x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


# model.summary()

87924736/87910968 [==============================] - 1s 0us/step


In [107]:
model.compile(Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_generator.reset()
validation_generator.reset()

# Fit the model
history = model.fit_generator(
    train_generator, 
    validation_data = validation_generator,
    epochs = 25,
    verbose=1)

In [112]:
labels = (train_generator.class_indices)
print(labels)

{'Centerstack': 0, 'Forward': 1, 'Left_wing_mirror': 2, 'Rearview_mirror': 3, 'Right_wing_mirror': 4}


In [113]:
labels = dict((i,j) for j,i in labels.items())
print(labels)

{0: 'Centerstack', 1: 'Forward', 2: 'Left_wing_mirror', 3: 'Rearview_mirror', 4: 'Right_wing_mirror'}


In [114]:
samp = pd.read_csv('/content/drive/MyDrive/Gaze_dataset/sample_submission.csv')

In [115]:
!ls

final1.csv	     haarcascade_frontalface_default.xml
final2.csv	     sample_submission.csv
final3.csv	     shape_predictor_68_face_landmarks.dat
final4.csv	     test
final.csv	     test_eye_cropped
haarcascade_eye.xml  train


In [116]:
for num,i in enumerate(samp['filename']):
  img = cv2.imread('test/'+i)
  img = cv2.resize(img, (224,224))
  img = img.reshape(-1, 224, 224, 3)
  prediction= model.predict(img)
  indices = np.argmax(prediction,axis=1)
  samp.loc[0, 'class']
  final_label = (labels[indices[0]])
  print(num,"  ",i,"   ",final_label)
  samp.loc[num, 'class'] = final_label

0    1.jpg     Forward
1    2.jpg     Forward
2    3.jpg     Forward
3    4.jpg     Forward
4    5.jpg     Forward
5    6.jpg     Forward
6    7.jpg     Forward
7    8.jpg     Forward
8    9.jpg     Forward
9    10.jpg     Forward
10    11.jpg     Forward
11    12.jpg     Forward
12    13.jpg     Forward
13    14.jpg     Forward
14    15.jpg     Forward
15    16.jpg     Forward
16    17.jpg     Forward
17    18.jpg     Forward
18    19.jpg     Forward
19    20.jpg     Forward
20    21.jpg     Forward
21    22.jpg     Forward
22    23.jpg     Forward
23    24.jpg     Forward
24    25.jpg     Forward
25    26.jpg     Forward
26    27.jpg     Forward
27    28.jpg     Forward
28    29.jpg     Forward
29    30.jpg     Forward
30    31.jpg     Forward
31    32.jpg     Forward
32    33.jpg     Forward
33    34.jpg     Forward
34    35.jpg     Forward
35    36.jpg     Forward
36    37.jpg     Forward
37    38.jpg     Forward
38    39.jpg     Forward
39    40.jpg     Forward
40    41.jpg     Fo

In [103]:
samp

,filename,class
0,1.jpg,Left_wing_mirror
1,2.jpg,Left_wing_mirror
2,3.jpg,Left_wing_mirror
3,4.jpg,Left_wing_mirror
4,5.jpg,Left_wing_mirror
...,...,...
504,505.jpg,Left_wing_mirror
505,506.jpg,Left_wing_mirror
506,507.jpg,Left_wing_mirror
507,508.jpg,Left_wing_mirror


In [104]:
samp.reset_index(drop=True).to_csv("final4.csv")